In [1]:
import sys
from pathlib import Path
from os.path  import join
from src.config import DATA_DIR, CONFIG_DIR
from src.external.hptr.src.data_modules.agent_centric import AgentCentricPreProcessing
from src.external.hptr.src.data_modules.ac_global import AgentCentricGlobal
from omegaconf import OmegaConf

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added

2025-01-31 18:23:52.627 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: G:\Work\DS\mimolm


g:\Work\DS\mimolm\notebooks


In [ ]:
ARGO_SCENE_DIR = DATA_DIR / "raw" / "temp"
NUM_SCENARIOS = 1

all_scenario_files = sorted(ARGO_SCENE_DIR.rglob("*.parquet"))
scenario_file_list = (
    all_scenario_files[:NUM_SCENARIOS]
)

from av2.datasets.motion_forecasting import scenario_serialization
from av2.map.map_api import ArgoverseStaticMap

scenario_path = scenario_file_list[0]
scenario_id = scenario_path.stem.split("_")[-1]
static_map_path = (
    scenario_path.parents[0] / f"log_map_archive_{scenario_id}.json"
)

scenario = scenario_serialization.load_argoverse_scenario_parquet(scenario_path)
static_map = ArgoverseStaticMap.from_json(static_map_path)

In [24]:
from av2.datasets.motion_forecasting.viz.scenario_visualization import visualize_scenario

visualize_scenario(scenario, static_map, DATA_DIR / "processed" / "argoext.mp4")

In [ ]:
import h5py

with h5py.File(DATA_DIR / "interim" / "train.h5", "r", libver="latest", swmr=True) as hf:
    print(hf['11']['agent'].keys())

In [2]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2

data_module = DataH5av2(DATA_DIR / "interim" / "train")
data_module.setup(stage="fit")
dl = data_module.train_dataloader()

c:\Users\Harshavardhan Patil\.virtualenvs\mimolm-kv68Yv56\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from torch import nn
from collections import OrderedDict
episode = None
preprocessor = nn.Sequential(OrderedDict([
('pre_1', AgentCentricPreProcessing(sampling_rate=5,
                        data_size=data_module.tensor_size_train, 
                        time_step_current=49, 
                        n_target=8,
                        n_other=48,
                        n_map=512,
                        mask_invalid=False)),
('pre_2', AgentCentricGlobal(sampling_rate=5,data_size=data_module.tensor_size_train,
                time_step_current=49,
                dropout_p_history=0.15, 
                add_ohe=True,
                pl_aggr=False,
                pose_pe= {"agent": "xy_dir",
                        "map": "mpa_pl"}))
                        ]))

In [5]:
ep = None

for batch in dl:
    ep = preprocessor(batch)
    break

In [7]:
ep['input/target_valid'].shape

torch.Size([2, 8, 25])

In [9]:
print(episode.keys())

dict_keys(['episode_idx', 'agent/valid', 'agent/pos', 'agent/vel', 'agent/spd', 'agent/acc', 'agent/yaw_bbox', 'agent/yaw_rate', 'agent/type', 'agent/cmd', 'agent/role', 'agent/size', 'agent/goal', 'agent/dest', 'map/valid', 'map/type', 'map/pos', 'map/dir', 'map/boundary', 'ref/idx', 'ref/idx_n', 'ref/pos', 'ref/rot', 'ref/type', 'ref/role', 'ac/target_valid', 'ac/target_pos', 'ac/target_vel', 'ac/target_spd', 'ac/target_acc', 'ac/target_yaw_bbox', 'ac/target_yaw_rate', 'ac/target_type', 'ac/target_role', 'ac/target_size', 'gt/valid', 'gt/spd', 'gt/pos', 'gt/vel', 'gt/yaw_bbox', 'gt/cmd', 'ac/other_valid', 'ac/other_spd', 'ac/other_acc', 'ac/other_yaw_rate', 'ac/other_pos', 'ac/other_vel', 'ac/other_yaw_bbox', 'ac/other_type', 'ac/other_role', 'ac/other_size', 'ac/map_valid', 'ac/map_type', 'ac/map_pos', 'ac/map_dir', 'input/target_type', 'input/target_valid', 'input/other_valid', 'input/map_valid', 'input/target_attr', 'input/other_attr', 'input/map_attr'])


In [1]:
%run -i ../src/external/hptr/src/pack_h5_av2.py
%tb

2025-01-30 20:19:04.005 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: G:\Work\DS\mimolm
  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (20,) + inhomogeneous part.

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (20,) + inhomogeneous part.

In [ ]:
#python -m scenarionet.convert_argoverse2 -d data\interim --raw_data_path G:\Work\DS\mimolm\data\raw\train --overwrite

In [12]:
import numpy as np
import torch


def read():
    arr = np.load('train.npy', allow_pickle=True)
    arr[()][0][0]
# with open('test.npy', 'wb') as f:
#     np.save(f, np.array([1, 2]))
#     np.save(f, np.array([1, 3]))
# with open('test.npy', 'rb') as f:
#     a = np.load(f)
#     b = np.load(f)
# print(a, b)

In [13]:
%timeit read()

103 ms ± 3.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
arr[()][0][0].shape

(1024, 20)

In [15]:
for k in arr[()][0].keys():
    print(arr[()][0][k].dtype)

int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64


In [27]:
from scenarionet import read_dataset_summary, read_scenario

dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path=DATA_DIR / "interim" / "train")

In [5]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=dataset, batch_size=3, num_workers=1, drop_last=False,
    collate_fn=dataset.collate_fn)

In [7]:
b = None
for batch in train_loader:
    print(batch.keys())
    b = batch
    break

dict_keys(['batch_size', 'input_dict', 'batch_sample_count'])


In [13]:
b['input_dict'].keys()

dict_keys(['center_gt_final_valid_idx', 'center_gt_trajs', 'center_gt_trajs_mask', 'center_gt_trajs_src', 'center_objects_id', 'center_objects_type', 'center_objects_world', 'dataset_name', 'kalman_difficulty', 'map_center', 'map_polylines', 'map_polylines_center', 'map_polylines_mask', 'obj_trajs', 'obj_trajs_future_mask', 'obj_trajs_future_state', 'obj_trajs_last_pos', 'obj_trajs_mask', 'obj_trajs_pos', 'scenario_id', 'track_index_to_predict', 'trajectory_type'])